<a href="https://colab.research.google.com/github/ealder04/ME485IMUProject/blob/master/nnTut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


In [ ]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

Processing the data. Data normalization could be helpful.

In [ ]:
dataframe = pandas.read_csv('https://github.com/ealder04/ME485IMUProject/raw/acb203bc63e3e2e40ad013d246a26e23003f0a40/M_Stroke', delim_whitespace=True, header=None)
Y = dataframe.values

dataframe = pandas.read_csv('https://github.com/ealder04/ME485IMUProject/raw/acb203bc63e3e2e40ad013d246a26e23003f0a40/StrokeStateFile', delim_whitespace=True, header=None)
X = dataframe.values

X0 = X[:,109:]
X = X[:,0:14]

X = np.concatenate((X,X0), axis=1)


In [ ]:
seed = 7
np.random.seed(seed)

# Set the number of neurons in the first and second layers (X0 and X1).
model = Sequential()
model.add(Dense(X0, input_dim=17, kernel_initializer='normal', activation='relu'))
# try different activation functions?
model.add(Dense(X1, kernel_initializer='normal', activation='relu'))
# add layers?
model.add(Dense(47, kernel_initializer='normal'))

model.compile(loss='mean_squared_error', optimizer='adam')

X, Y = shuffle_in_unison(X,Y)

In [ ]:
split_size = int(len(X) * 0.8)

X_train, X_val, X_test = np.split(X, [int(.8 * len(X)), int(.9 * len(X))])
Y_train, Y_val, Y_test = np.split(Y, [int(.8 * len(Y)), int(.9 * len(Y))])

early_stopping = EarlyStopping(monitor='val_loss', patience=30)
model.fit(X_train,Y_train, epochs=1000, batch_size=100, verbose=2, validation_data=(X_val, Y_val), callbacks=[early_stopping])

Epoch 1/1000
78/78 - 1s - loss: 0.3755 - val_loss: 0.3717
Epoch 2/1000
78/78 - 0s - loss: 0.3430 - val_loss: 0.3502
Epoch 3/1000
78/78 - 0s - loss: 0.3209 - val_loss: 0.3317
Epoch 4/1000
78/78 - 0s - loss: 0.3058 - val_loss: 0.3178
Epoch 5/1000
78/78 - 0s - loss: 0.2885 - val_loss: 0.3090
Epoch 6/1000
78/78 - 0s - loss: 0.2767 - val_loss: 0.2921
Epoch 7/1000
78/78 - 0s - loss: 0.2642 - val_loss: 0.2775
Epoch 8/1000
78/78 - 0s - loss: 0.2526 - val_loss: 0.2691
Epoch 9/1000
78/78 - 0s - loss: 0.2460 - val_loss: 0.2609
Epoch 10/1000
78/78 - 0s - loss: 0.2334 - val_loss: 0.2548
Epoch 11/1000
78/78 - 0s - loss: 0.2266 - val_loss: 0.2557
Epoch 12/1000
78/78 - 0s - loss: 0.2204 - val_loss: 0.2414
Epoch 13/1000
78/78 - 0s - loss: 0.2114 - val_loss: 0.2306
Epoch 14/1000
78/78 - 0s - loss: 0.2045 - val_loss: 0.2228
Epoch 15/1000
78/78 - 0s - loss: 0.2013 - val_loss: 0.2163
Epoch 16/1000
78/78 - 0s - loss: 0.1941 - val_loss: 0.2164
Epoch 17/1000
78/78 - 0s - loss: 0.1927 - val_loss: 0.2075
Epoch 

Comparing the model prediction with the test data on one example: 

In [ ]:
predictions = model.predict(X_test)
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
print(predictions[1,:]) 
print(Y_test[1,:])

[1.363 -0.034 0.743 0.139 -0.525 0.362 -0.352 1.009 -0.893 0.114 -0.343
 -0.014 -0.470 0.259 -0.527 0.137 -0.323 0.579 -0.903 -0.199 -0.567 -0.151
 -0.379 0.250 -0.517 0.869 -1.189 -0.411 -0.189 0.486 1.021 1.132 -0.313
 -0.573 -0.402 -0.944 -0.749 0.323 -0.496 -0.678 -1.150 -0.661 0.392
 -0.800 -0.468 0.795 -0.387]
[1.322 -0.220 0.763 0.227 -0.713 0.493 -0.368 1.147 -0.929 0.158 -0.305
 0.122 -0.530 0.043 -0.376 -0.027 -0.279 0.469 -1.090 -0.018 -0.763 -0.068
 -0.317 0.499 -0.394 0.991 -1.312 -0.231 -0.117 0.617 1.032 1.426 -0.105
 -0.664 -0.349 -1.138 -0.769 0.417 -0.446 -0.913 -1.253 -0.749 0.677
 -0.894 -0.701 0.845 -0.636]


How does the model perform on the test set?

In [ ]:
results = model.evaluate(X_test, Y_test)
print("test loss", results)

31/31 [==============================] - 0s 2ms/step - loss: 0.0804
test loss, test acc: 0.08043497055768967
